In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [20]:
train_data = pd.read_csv('Titanic/train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
test_data = pd.read_csv('Titanic/test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [22]:
# Calculate the percentage of survivors by gender
survival_by_sex = train_data.groupby('Sex')['Survived'].mean() * 100

print(f"% of women who survived: {survival_by_sex['female']:.2f}%")
print(f"% of men who survived: {survival_by_sex['male']:.2f}%")

% of women who survived: 74.20%
% of men who survived: 18.89%


In [23]:
train_data['Survived'].value_counts()
#train_data['Survived'].value_counts(normalize=True) * 100

Survived
0    549
1    342
Name: count, dtype: int64

In [24]:
# Get count of survived men
men_survived = train_data[(train_data['Sex'] == 'male') & (train_data['Survived'] == 1)]
print("Number of men who survived:", len(men_survived))

# Get count of survived women
women_survived = train_data[(train_data['Sex'] == 'female') & (train_data['Survived'] == 1)]
print("Number of women who survived:", len(women_survived))

Number of men who survived: 109
Number of women who survived: 233


In [25]:
def preprocess(data):
    data = data.copy()
    
    # Fill missing Age and Fare with median
    imputer = SimpleImputer(strategy='median')
    data['Age'] = imputer.fit_transform(data[['Age']])
    if 'Fare' in data.columns:
        data['Fare'] = imputer.fit_transform(data[['Fare']])
    
    # Fill missing Embarked
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
    
    # Encode categorical columns
    label = LabelEncoder()
    data['Sex'] = label.fit_transform(data['Sex'])
    data['Embarked'] = label.fit_transform(data['Embarked'])
    
    return data

train_processed = preprocess(train_data)
test_processed = preprocess(test_data)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_17004\1719280963.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_17004\1719280963.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

In [26]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = train_processed[features]
y = train_processed['Survived']

In [27]:
rf = RandomForestClassifier(random_state=42)
rf_score = cross_val_score(rf, X, y, cv=5).mean()
print(f"Random Forest CV Accuracy: {rf_score:.4f}")

Random Forest CV Accuracy: 0.8092


In [28]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_score = cross_val_score(xgb, X, y, cv=5).mean()
print(f"XGBoost CV Accuracy: {xgb_score:.4f}")

c:\Titanic_classification- project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Titanic_classification- project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Titanic_classification- project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost CV Accuracy: 0.8182


c:\Titanic_classification- project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Titanic_classification- project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [29]:
svm = SVC()
svm_score = cross_val_score(svm, X, y, cv=5).mean()
print(f"SVM CV Accuracy: {svm_score:.4f}")

SVM CV Accuracy: 0.6746


In [30]:
scores = {'RandomForest': rf_score, 'XGBoost': xgb_score, 'SVM': svm_score}
best_model_name = max(scores, key=scores.get)
print("Best model is:", best_model_name)

if best_model_name == 'RandomForest':
    best_model = rf
elif best_model_name == 'XGBoost':
    best_model = xgb
else:
    best_model = svm

best_model.fit(X, y)

Best model is: XGBoost


c:\Titanic_classification- project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [31]:
X_test = test_processed[features]
final_prediction = best_model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': final_prediction})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
